In [37]:
# Data
import numpy as np
import pandas as pd

# NLP
import re

from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

# Preprocesamiento
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


In [38]:
data = pd.read_csv('spam.csv', encoding='latin-1')

In [39]:
data

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [40]:
data.drop([data.columns[col] for col in [2, 3, 4]], axis=1, inplace=True)

In [41]:
#Limpiamos los datos, los tokenizamos y los regresamos como una lista de palabras
def processEmail(contents):
    ps = PorterStemmer()
    
    contents = contents.lower()
    contents = re.sub(r'<[^<>]+>', ' ', contents)
    contents = re.sub(r'[0-9]+', 'number', contents)
    contents = re.sub(r'(http|https)://[^\s]*', 'httpaddr', contents)
    contents = re.sub(r'[^\s]+@[^\s]+', 'emailaddr', contents)
    contents = re.sub(r'[$]+', 'dollar', contents)
    
    words = word_tokenize(contents)
    
    for i in range(len(words)):
        words[i] = re.sub(r'[^a-zA-Z0-9]', '', words[i])
        words[i] = ps.stem(words[i])
        
    words = [word for word in words if len(word) >= 1]
    
    return words

In [42]:
#Recibimos la lista de las palabras usadas en todos los emails 
# y creamos un vocabulario con las palabras más frecuentes
def getVocabulary(emails, vocab_length):
    vocabulary = dict()
    
    for i in range(len(emails)):
        emails[i] = processEmail(emails[i])
        for word in emails[i]:
            if word in vocabulary.keys():
                vocabulary[word] += 1
            else:
                vocabulary[word] = 1
                
    vocabulary = sorted(vocabulary.items(), key=lambda x: x[1], reverse=True)
    vocabulary = list(map(lambda x: x[0], vocabulary[0:vocab_length]))
    vocabulary = {index: word for index, word in enumerate(vocabulary)}
    
    return vocabulary

In [43]:
# Recorrimos el diccionario buscando la clave de un valor específico
# Esto es para el modelo de Machine Learning
def getKey(dictionary, val):
    for key, value in dictionary.items():
        if value == val:
            return key

In [44]:
# Devolvemos los índices de las palabras del vocabulario que aparecen en un email
# Esto es para el modelo de Machine Learning
def getIndices(email, vocabulary):
    word_indices = set()
    
    for word in email:
        if word in vocabulary.values():
            word_indices.add(getKey(vocabulary, word))
    
    return word_indices

In [45]:
# Esto es para el modelo de Machine Learning
def getFeatureVector(word_indices, vocab_length):
    feature_vec = np.zeros(vocab_length)
    
    for i in word_indices:
        feature_vec[i] = 1
        
    return feature_vec

In [46]:
vocab_length = 2000

In [47]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sophiagabrielamartinezalbarran/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/sophiagabrielamartinezalbarran/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [48]:
# Construímos el vocabulario con las palabras más frecuentes
vocabulary = getVocabulary(data['v2'].to_list(), vocab_length)

# Limpiamos y tokenizamos todos los correos
emails = data['v2'].to_list()
emails = list(map(lambda x: processEmail(x), emails))

In [49]:
# El vocabulario es un diccionario con índices y palabras
vocabulary

{0: 'i',
 1: 'number',
 2: 'to',
 3: 'you',
 4: 'a',
 5: 'the',
 6: 'u',
 7: 'and',
 8: 'it',
 9: 'is',
 10: 'in',
 11: 'me',
 12: 'my',
 13: 'for',
 14: 'your',
 15: 'call',
 16: 'have',
 17: 'do',
 18: 'that',
 19: 'of',
 20: 's',
 21: 'on',
 22: 'are',
 23: 'now',
 24: 'so',
 25: 'go',
 26: 'get',
 27: 'not',
 28: 'but',
 29: 'be',
 30: 'or',
 31: 'm',
 32: 'can',
 33: 'at',
 34: 'we',
 35: 'will',
 36: 'if',
 37: 'ur',
 38: 'with',
 39: 'nt',
 40: 'just',
 41: 'no',
 42: 'thi',
 43: 'how',
 44: 'gt',
 45: 'lt',
 46: 'up',
 47: 'what',
 48: 'come',
 49: 'when',
 50: 'ok',
 51: 'from',
 52: 'free',
 53: 'know',
 54: 'all',
 55: 'out',
 56: 'like',
 57: 'got',
 58: 'love',
 59: 'day',
 60: 'time',
 61: 'wa',
 62: 'want',
 63: 'good',
 64: 'then',
 65: 'll',
 66: 'there',
 67: 'he',
 68: 'text',
 69: 'am',
 70: 'onli',
 71: 'send',
 72: 'hi',
 73: 'need',
 74: 'one',
 75: 'txt',
 76: 'as',
 77: 'today',
 78: 'see',
 79: 'by',
 80: 'take',
 81: 'think',
 82: 'about',
 83: 'she',
 84: 'd

In [50]:
# Devolvemos los índices de las palabras del vocabulario que aparecen en un email
getIndices('email', vocabulary)

{0, 4, 31, 170}

### Esto es para el entrenamiento de un modelo de Machine Learning 

In [51]:
X = list(map(lambda x: getFeatureVector(getIndices(x, vocabulary), vocab_length), emails))
X = pd.DataFrame(np.array(X).astype(np.int16))

In [52]:
y = data['v1']

In [53]:

class NativeBayes():
    def __init__(self):
        self.probabilidad_clase = dict()
        self.probabilidad_palabra = dict()
        self.clases = []
    
    def fit(self, X, y):
        # Obtenemos las clases únicas, en este caso spam y ham
        self.clases = np.unique(y)
        
        # Guardamos la probabilidad de cada clase y la probabilidad de cada palabra dado una clase en un diccionario para luego poder hacer las predicciones
        for c in self.clases:
            X_c = X[y == c]
            self.probabilidad_clase[c] = X_c.shape[0] / X.shape[0]
            
            # Suavizado de Laplace: Se usa para evitar probabilidades de cero, ya que si predecimos un nuevo email que no contiene una palabra del vocabulario 
            # la probabilidad de esa palabra sería cero, lo cual afectaría el cálculo de la probabilidad total por lo que asumimos que todas las palabras
            # al menos aparecen una vez en cada clase
            self.probabilidad_palabra[c] = (X_c.sum(axis=0) + 1) / (X_c.shape[0] + 2)
    
    def predecir(self, X):
        predicciones = []
        
        # Predecimos la clase para cada email
        for i in range(X.shape[0]):
            prediciones_por_clase = dict()
            
            # Probabilidad a priori de cada clase
            for c in self.clases:
                # Vamos a usar logaritmos para evitar underflow y para que las probabilidades sean más manejables, tambien para evitar
                # multiplicar y mejor sumar, log(a x b) = log(a) + log(b)
                probabilidad_clase_c = np.log(self.probabilidad_clase[c])
                
                # Tomamos en cuenta la probabilidad de cada palabra en el email dado la clase, pero también la probabilidad de que no aparezca
                # ya que ambas cosas nos dan información sobre la clase del email
                probabilidad = X.iloc[i] * np.log(self.probabilidad_palabra[c]) + (1 - X.iloc[i]) * np.log(1 - self.probabilidad_palabra[c])
                probabilidad = probabilidad.sum()
                probabilidad_final = probabilidad_clase_c + probabilidad
                prediciones_por_clase[c] = probabilidad_final
            
            # Tomamos la clase con la probabilidad más alta, es decir si P(clase|email) es mayor para spam que para ham, entonces el email es spam
            predicciones.append(max(prediciones_por_clase, key=prediciones_por_clase.get))
        
        return np.array(predicciones)

    def predict_proba(self, X):
        """
        Retorna las probabilidades REALES de cada clase para cada email
        Usa la fórmula completa: P(clase|email) = P(email|clase) × P(clase) / P(email)
        """
        probabilidades = []
        
        for i in range(X.shape[0]):
            # Calculamos P(email|clase) × P(clase) para cada clase
            numeradores = dict()
            
            for c in self.clases:
                # P(clase) - probabilidad a priori
                #Es la proporción de correos spam o ham en el dataset.
                #La usamos porque Bayes dice que cada clase tiene un peso inicial antes de ver el email.
                prior = self.probabilidad_clase[c]
                
                # P(email|clase) - likelihood
                # Si la palabra aparece en el email entonces se va a multiplicar P(palabra|clase)
                # Si la palabra no aparece en el email se multiplica por (1 - P(palabra|clase))
                likelihood = 1.0
                for j in range(len(X.iloc[i])):
                    if X.iloc[i].iloc[j] == 1:  # palabra presente
                        likelihood *= self.probabilidad_palabra[c].iloc[j]
                    else:  # palabra ausente
                        likelihood *= (1 - self.probabilidad_palabra[c].iloc[j])
                
                # P(email|clase) × P(clase)
                #Es la probabilidad conjunta de que el email y la clase ocurran juntos.
                numeradores[c] = likelihood * prior
            
            # P(email) = suma de todos los numeradores
            #Es la probabilidad total del email sin importar la clase.
            #Se necesita para normalizar y que las probabilidades finales sumen 1.
            denominador = sum(numeradores.values())
            
            # P(clase|email) = P(email|clase) × P(clase) / P(email)
            # Esto nos da la probabilidad de que el email pertenezca a cada clase (Spam o Ham)
            probabilidades_reales = [numeradores[c] / denominador for c in sorted(self.clases)]
            probabilidades.append(probabilidades_reales)
            
        return np.array(probabilidades)

In [54]:
# Ejemplo de uso de predict_proba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo
modelo = NativeBayes()
modelo.fit(X_train, y_train)

# Predicciones de clase
predicciones = modelo.predecir(X_test.head(5))
print("Predicciones de clase:", predicciones)

# Probabilidades
probabilidades = modelo.predict_proba(X_test.head(5))
print("Probabilidades [P(ham), P(spam)]:")
for i, probs in enumerate(probabilidades):
    print(f"Email {i}: P(ham)={probs[0]:.4f}, P(spam)={probs[1]:.4f}")

y_test.head(5)

Predicciones de clase: ['ham' 'ham' 'spam' 'ham' 'spam']
Probabilidades [P(ham), P(spam)]:
Email 0: P(ham)=0.9991, P(spam)=0.0009
Email 1: P(ham)=0.9994, P(spam)=0.0006
Email 2: P(ham)=0.0000, P(spam)=1.0000
Email 3: P(ham)=1.0000, P(spam)=0.0000
Email 4: P(ham)=0.0000, P(spam)=1.0000


3245     ham
944      ham
1044    spam
2484     ham
812     spam
Name: v1, dtype: object